# 自定义线性模型 - 2

## 1. 导入包

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

## 2. 定义变量和placeholder

In [ ]:
W = tf.Variable(0, dtype=tf.float64, name='W')
b = tf.Variable(0, dtype=tf.float64, name='b')

x = tf.placeholder(dtype=tf.float64, name='x')
y = tf.placeholder(dtype=tf.float64, name='y')

## 3. 准备数据

In [ ]:
# y = 4x + 5
x_train = [1, 2]
y_train = [9, 13]

# 学习率
learning_rate = 0.01

N = float(len(x_train))

# 迭代总次数
epochs = 3000
# 保存每次迭代loss值
metrics = []
# 保存每次迭代W值
W_list = []

# 保存每次迭代b值
b_list = []

W_gradient_list = []
b_gradient_list = []


## 4. 线性模型和损失函数

In [ ]:
# 设计模型
model = x * W + b

# 计算损失函数 - mean squared error (MSE)
loss = tf.sqrt(tf.reduce_sum(tf.square(model - y)))
tf.summary.scalar('loss', loss)

## 5. 最小二乘法

$$\frac{\partial Loss} {\partial  W} = \frac{ \frac {1} {N} \sum _{ i = 0 } ^N (y_{i} - (Wx_{i} + b))^2 } {\partial W} = -\frac {2} {N} \sum_{i=1} ^N x_{i}(y_{i} - (Wx_{i} + b)) = 0 $$

$$\frac{\partial Loss} {\partial  b} = \frac{ \frac {1} {N} \sum _{ i = 0 } ^N (y_{i} - (Wx_{i} + b))^2 } {\partial b} = -\frac {2} {N} \sum_{i=1} ^N (y_{i} - (Wx_{i} + b)) = 0 $$

$$model = Wx_{i} + b$$

In [ ]:
# y - model决定方向
# Σ -2/N * x * (y - (W * x + b)) -> 下面的迭代可以将这个值逼向0
W_gradient = -2/N * tf.reduce_sum((y - model) * x)
# Σ -2/N * (y - (x * W + b))  ->  下面的迭代可以将这个值逼向0
b_gradient = -2/N * tf.reduce_sum(y - model)

## 6. 梯度下降

<img src="./images/gradient.png" />

In [ ]:
# 梯度下降
new_b = b.assign(b - learning_rate * b_gradient)
new_W = W.assign(W - learning_rate * W_gradient)

## 7. 初始化变量

tf在使用变量之前，需要通过该方法先初始化变量

In [ ]:
# 初始化变量
init = tf.global_variables_initializer()

## 8. 训练模型

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(epochs):        
        new_W_r, new_b_r,W_gradient_r, b_gradient_r, loss_r = sess.run(
            [
                new_W, 
                new_b, 
                W_gradient,
                b_gradient,
                loss                
            ], 
            feed_dict={
                x: x_train, 
                y: y_train
            }
        )
        
        metrics.append(loss_r)
        W_list.append(new_W_r)
        b_list.append(new_b_r)
        
        W_gradient_list.append(W_gradient_r)
        b_gradient_list.append(b_gradient_r)
        
    # 使用模型预测
    result = sess.run(model, feed_dict={x: 5})
    print(result)

## 9. 图形显示迭代过程值的变化

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot([i + 1 for i in range(len(W_list))], W_list, 'r', [i + 1 for i in range(len(b_list))], b_list, 'b')

plt.plot([4] * len(W_list), 'r--', [5] * len(b_list), 'b--')

plt.legend(['W', 'b', 'true W', 'true_b'])

plt.figure(figsize=(15, 3))
plt.plot([i + 1 for i in range(len(metrics))], metrics, label='mean squared error')
plt.xlabel('epochs')
plt.ylabel('MSE')
plt.legend()

plt.show()


## 10. 图形方式显示梯度变化

In [ ]:
start = 80
plt.figure(figsize=(15, 3))
plt.plot([i for i in range(len(W_gradient_list) - start)], W_gradient_list[start:], label='W_gradient_list')
plt.xlabel('epochs')
plt.ylabel('W_gradient')
plt.show()

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot([i for i in range(len(b_gradient_list) - start)], b_gradient_list[start:], label='b_gradient_list')
plt.xlabel('epochs')
plt.ylabel('b_gradient')
plt.show()